In [2]:
import os
os.chdir(os.path.abspath(os.path.join(os.getcwd() ,"..")))

In [39]:
import joblib
import warnings
import src.data.sql_functions as sql
import src.data.data_transformation as cln
import pandas_ta as pta
from sklearn.preprocessing import RobustScaler

In [35]:
# Global variables
warnings.filterwarnings("ignore")
DB_SERVER   = "tcp:AUCLD04018656,1433"
DB_NAME     = "team_five_aiml_group"
select_features = ['activeaddresses_btc', 'bitcoin_confirmationtime', 'bitcoin_difficulty',
       'bitcoin_transaction', 'bitcoin_transactionfees', 'fee_to_reward_btc',
       'google_trends_btc', 'mediantransactionvalue_btc', 'mkt_Volume',
       'sentinusd_btc', 't_closing_roc', 't_closing_rsi', 't_closing_var',
       't_range', 'top100cap_btc', 'transactionvalue_btc', 'tweets_btc']
LIST_CONVERT_FLOAT = [
'bitcoin_transaction'                       
,'size_btc'                                  
,'sentbyaddress_btc'                         
,'bitcoin_difficulty'                        
,'bitcoin_hashrate'                          
,'bitcoin_mining_profitability'              
,'sentinusd_btc'                             
,'bitcoin_transactionfees'                   
,'bitcoin_median_transaction_fee'            
,'bitcoin_confirmationtime'                  
,'transactionvalue_btc'                      
,'mediantransactionvalue_btc'                
,'tweets_btc'                                
,'google_trends_btc'                         
,'activeaddresses_btc'                       
,'top100cap_btc'                             
,'fee_to_reward_btc'] 
SMOOTHING_RANGE = 5

In [12]:
# Later you can load it like this:
model = joblib.load('models/test_btc.mod')

In [15]:
# Get market/supp data from database
conn = sql.SqlDb(DB_SERVER, DB_NAME)
get_query = "SELECT * FROM [{db}].[dbo].BTC_AUD_{dt} A LEFT JOIN [{db}].[dbo].BTC_SUPP_DATA_{dt} B ON A.mkt_Date = B.supp_date".format(db=DB_NAME, dt="20130101_20230416")
df_btc_input = conn.query(get_query, True)
conn.__del__

<bound method SqlDb.__del__ of <src.data.sql_functions.SqlDb object at 0x000001CC3C541CA0>>

In [44]:

df_btc_input['t_plus_1_Close'] = df_btc_input['mkt_Close'].shift(-1)
df_btc_input['t_plus_1_Return'] = df_btc_input['mkt_Close'].pct_change(1)
df_btc_input['t_range'] = df_btc_input['mkt_High'] - df_btc_input['mkt_Low'] 
df_btc_input['t_closing_var'] = pta.variance(df_btc_input['mkt_Close'], SMOOTHING_RANGE)
df_btc_input['t_closing_rsi'] = pta.rsi(df_btc_input['mkt_Close'], SMOOTHING_RANGE)
df_btc_input['t_closing_roc'] = pta.roc(df_btc_input['mkt_Close'],SMOOTHING_RANGE)

for c in LIST_CONVERT_FLOAT:
     df_btc_input[c] = df_btc_input[c].astype(float) 

clean_df_btc_input = cln.clean_df(df_btc_input)

scaler = RobustScaler()
x = clean_df_btc_input
x[select_features] = scaler.fit_transform(x[select_features])

In [45]:
inp_feat = x[x['mkt_Date'] == '2023-04-14']

In [46]:
pred_return = model.predict(inp_feat[select_features])[0]

In [55]:
print("Our prediction for 2023-04-15 is: {}".format((1 + pred_return) * clean_df_btc_input[clean_df_btc_input['mkt_Date'] == '2023-04-14']['mkt_Close']))
print("But, the actual closing price was: {}".format(clean_df_btc_input[clean_df_btc_input['mkt_Date'] == '2023-04-15']['mkt_Close']))

Our prediction for 2023-04-15 is: 3131    45695.394519
Name: mkt_Close, dtype: float64
But, the actual closing price was: 3132    45460.335938
Name: mkt_Close, dtype: float64


0.019737703221213657